In [92]:
import torch
import torchvision
from torchvision.datasets import MNIST
import numpy as np

In [93]:
dataset=MNIST(root='data/',download=True)

In [94]:
len(dataset)

60000

In [95]:
test_dataset=MNIST(root='data/',train=False)
len(test_dataset)

10000

In [96]:
dataset[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x1A273422148>, 5)

In [97]:
image,label=dataset[0]


In [98]:
import torchvision.transforms as transforms

In [99]:
dataset=MNIST(root='data/',train=True,transform=transforms.ToTensor())

In [100]:
img_tensor, label =dataset[0]
print(img_tensor.shape,label)

torch.Size([1, 28, 28]) 5


In [101]:
print(img_tensor[:,10:15,10:15])
print(torch.max(img_tensor),torch.min(img_tensor))

tensor([[[0.0039, 0.6039, 0.9922, 0.3529, 0.0000],
         [0.0000, 0.5451, 0.9922, 0.7451, 0.0078],
         [0.0000, 0.0431, 0.7451, 0.9922, 0.2745],
         [0.0000, 0.0000, 0.1373, 0.9451, 0.8824],
         [0.0000, 0.0000, 0.0000, 0.3176, 0.9412]]])
tensor(1.) tensor(0.)


In [102]:
n=6000
val_pct=0.2
n_val=int(val_pct*n)
idxs=np.random.permutation(n)
idxs


array([4516, 4460,  873, ..., 3571,  295, 3317])

In [103]:


def split_indices(n, val_pct):
    n_val=int(val_pct*n)
    idxs=np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

In [104]:
train_indices, val_indices=split_indices(len(dataset),val_pct=0.2)

In [105]:
print(len(train_indices),len(val_indices))
print('Sample val indices: ',val_indices[:20])

48000 12000
Sample val indices:  [27726 21489 34116 41909 53429 49901 15926 39981 14552 20674 40225   972
  7136 25595 14052 34462 57341 10408 52883 55604]


In [106]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [107]:
batch_size=100

train_sampler=SubsetRandomSampler(train_indices)
train_loader=DataLoader(dataset,batch_size,sampler=train_sampler)

val_sampler=SubsetRandomSampler(val_indices)
val_loader=DataLoader(dataset,batch_size,sampler=val_sampler)

In [108]:
import torch.nn as nn
input_size=28*28
num_classes=10

model=nn.Linear(input_size,num_classes)

In [109]:
print(model.weight.shape)
model.weight

torch.Size([10, 784])


Parameter containing:
tensor([[ 0.0327, -0.0287, -0.0262,  ...,  0.0298, -0.0104, -0.0352],
        [ 0.0199, -0.0134, -0.0053,  ...,  0.0277, -0.0147,  0.0149],
        [ 0.0205, -0.0199,  0.0149,  ..., -0.0064, -0.0338, -0.0046],
        ...,
        [-0.0243, -0.0114, -0.0047,  ...,  0.0028, -0.0035,  0.0123],
        [-0.0143, -0.0203, -0.0314,  ..., -0.0244, -0.0051,  0.0229],
        [ 0.0306,  0.0193,  0.0060,  ..., -0.0292,  0.0003, -0.0155]],
       requires_grad=True)

In [110]:
print(model.bias.shape)
model.bias

torch.Size([10])


Parameter containing:
tensor([ 0.0296,  0.0196,  0.0252,  0.0133, -0.0255,  0.0292, -0.0191, -0.0113,
         0.0276,  0.0175], requires_grad=True)

In [111]:
for images, labels in train_loader:
    print(labels)
    print(images.shape)
    outputs=model(images)
    break

tensor([2, 2, 8, 6, 3, 2, 4, 7, 8, 6, 0, 5, 5, 6, 1, 1, 7, 4, 3, 0, 6, 5, 7, 9,
        7, 4, 9, 4, 1, 9, 2, 3, 7, 3, 4, 9, 8, 4, 8, 4, 3, 1, 1, 1, 4, 1, 4, 9,
        4, 5, 7, 0, 9, 1, 6, 5, 5, 5, 9, 3, 9, 8, 2, 8, 8, 7, 9, 1, 6, 6, 5, 9,
        2, 8, 5, 8, 9, 9, 4, 3, 8, 8, 8, 3, 3, 5, 9, 1, 3, 7, 1, 0, 8, 6, 2, 8,
        1, 7, 5, 8])
torch.Size([100, 1, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2800x28 and 784x10)

In [ ]:
class MnistModel(nn.Module):

    def __init__(self):

        super().__init__()
        self.linear=nn.Linear(input_size,num_classes)

    def forward(self,xb):
        
        xb=xb.reshape(-1,784)
        out=self.linear(xb)
        return out
    
model=MnistModel()

In [115]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())

AttributeError: 'Linear' object has no attribute 'linear'